# "COVID-19 en Chile: Estadísticas demográficas"
> Total de casos confirmados, fallecidos confirmados, pacientes en UCI por sexo y rango de edad.

- toc: true 
- badges: true
- comments: true
- author: Alonso Silva Allende
- categories: [jupyter]
- image: images/diagram.png

In [22]:
#hide
import numpy as np
import pandas as pd
import altair as alt
from IPython.display import display, Markdown, display_html, HTML

In [23]:
#hide
update_date = pd.to_datetime('today') - pd.offsets.Hour(1)

In [24]:
#hide_input
display(Markdown(f"Última actualización: {update_date.strftime('%d/%m/%Y')}."))

Última actualización: 19/03/2021.

# Evolución de casos confirmados por sexo

In [25]:
#hide
cases_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/' \
    + 'master/output/producto16/CasosGeneroEtario.csv'
)

In [26]:
#hide
#first and last report date
first_date = cases_raw.T.iloc[2].name
last_date = cases_raw.T.iloc[-1].name
first_date, last_date

('2020-03-25', '2021-03-15')

In [27]:
#hide
cases_sex = cases_raw.groupby('Sexo').sum()
cases_sex = cases_sex.reset_index().melt('Sexo', var_name="Fecha (año-mes-día)", value_name="Casos confirmados")

In [28]:
#hide
cases_sex = cases_sex.set_index("Fecha (año-mes-día)")
cases_sex["Porcentaje"] = cases_sex["Casos confirmados"]/cases_raw.sum().drop(["Grupo de edad", "Sexo"])
cases_sex = cases_sex.reset_index()

In [29]:
#hide_input
input_dropdown = alt.binding_select(options=cases_sex['Sexo'].unique())
selection1 = alt.selection_single(fields=['Sexo'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['Sexo'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('Sexo:N', scale=alt.Scale(scheme='tableau10'), legend=None),
                    alt.value('lightgray'))

bars1 = alt.Chart(cases_sex).mark_bar().encode(
    x=alt.X("Fecha (año-mes-día)", axis=alt.Axis(title="")),
    y=alt.Y("Casos confirmados"),
    tooltip = ["Fecha (año-mes-día)", "Casos confirmados", "Porcentaje"],
    color=color
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

legend = alt.Chart(cases_sex).mark_point().encode(
    y=alt.Y('Sexo:N', axis=alt.Axis(orient='right')),
    color=color
).add_selection(
    selection1, selection2
)

bars2 = alt.Chart(cases_sex).mark_bar().encode(
    x=alt.X("Fecha (año-mes-día)", axis=alt.Axis(title="Fecha (año-mes-día)", ticks=False, labels=False)),
    y=alt.Y("Casos confirmados", axis=alt.Axis(title="Normalizados", format='%'), stack="normalize"),
    color=color,
    tooltip = ["Fecha (año-mes-día)", "Casos confirmados", "Porcentaje"],
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

alt.vconcat(
    bars1.properties(
        title = 'COVID-19 en Chile: Evolución de casos confirmados por sexo',
        width=600) | legend,
    bars2.properties(
        height = 80,
        width=600)
).configure_view(
    stroke=None
).configure_concat(
    spacing=1
)

alt.VConcatChart(...)

In [30]:
#hide
#build dataframe with all the dates since first date to last date
cases_by_sex = pd.DataFrame()
cases_by_sex["Fecha"] = pd.date_range(start=first_date, end=last_date).strftime('%Y-%m-%d')
cases_by_sex = cases_by_sex.set_index("Fecha")

In [31]:
#hide
cases_by_sex[["Mujeres", "Hombres"]] = cases_raw.groupby('Sexo').sum().T

In [32]:
#hide
cases_by_sex_long_form = cases_by_sex.reset_index().melt('Fecha', var_name='Sexo', value_name='Casos confirmados')

In [33]:
#hide
cases_by_sex_long_form = cases_by_sex_long_form.set_index("Fecha")

In [34]:
#hide
cases_by_sex_long_form["Porcentaje"] = \
(cases_by_sex_long_form["Casos confirmados"]/cases_raw.sum().drop(["Grupo de edad", "Sexo"])).values

In [35]:
#hide
cases_by_sex_long_form.head()

,Sexo,Casos confirmados,Porcentaje
Fecha,,,
2020-03-25,Mujeres,516.0,0.509881
2020-03-26,Mujeres,642.0,0.490119
2020-03-27,Mujeres,730.0,0.51278
2020-03-28,Mujeres,856.0,0.48722
2020-03-29,Mujeres,947.0,0.509066


In [36]:
#hide
input_dropdown = alt.binding_select(options=cases_by_sex_long_form['Sexo'].unique())
selection1 = alt.selection_single(fields=['Sexo'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['Sexo'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('Sexo:N', scale=alt.Scale(scheme='tableau10'), legend=None),
                    alt.value('lightgray'))

bars1 = alt.Chart(cases_by_sex_long_form.reset_index()).mark_bar().encode(
    x=alt.X("Fecha", axis=alt.Axis(title="")),
    y=alt.Y("Casos confirmados"),
    tooltip = ["Fecha", "Casos confirmados"],
    color=color
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

legend = alt.Chart(cases_by_sex_long_form.reset_index()).mark_point().encode(
    y=alt.Y('Sexo:N', axis=alt.Axis(orient='right')),
    color=color
).add_selection(
    selection1, selection2
)

bars2 = alt.Chart(cases_by_sex_long_form.reset_index()).mark_bar().encode(
    x=alt.X("Fecha", axis=alt.Axis(title="Fecha", ticks=False, labels=False)),
    y=alt.Y("Casos confirmados", axis=alt.Axis(title="Normalizados", format='%'), stack="normalize"),
    color=color,
    tooltip = ["Fecha", "Casos confirmados"],
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

alt.vconcat(
    bars1.properties(
        title = 'COVID-19 en Chile: Evolución de casos confirmados por sexo',
        width=600) | legend,
    bars2.properties(
        height = 80,
        width=600)
).configure_view(
    stroke=None
).configure_concat(
    spacing=1
)

alt.VConcatChart(...)

In [37]:
#hide
#country_population = pd.read_csv("http://www.ine.cl/docs/default-source/proyecciones-de-poblacion/cuadros-estadisticos/base-2017/ine_estimaciones-y-proyecciones-de-poblaci%C3%B3n-1992-2050_base-2017_base-de-datos.csv",
#                         encoding='ISO-8859-1', sep=";", header=1, thousands='.')

In [38]:
#hide
#country_population.query("EDAD == 'TOTAL'")

In [39]:
#hide
#pop_hombres = country_population.loc[206, "2020"] # 9599101
#pop_mujeres = country_population.loc[311, "2020"] # 9859209
#pop_hombres, pop_mujeres

In [40]:
#hide
pop_hombres, pop_mujeres = 9599101, 9859209

In [41]:
#hide
cases_sex = cases_raw.groupby('Sexo').sum()

In [42]:
#hide
cases_sex[last_date]

Sexo
F    514356
M    510658
Name: 2021-03-15, dtype: int64

In [43]:
#hide
casos_hombres = cases_sex.loc["M",last_date]
casos_mujeres = cases_sex.loc["F",last_date]
casos_hombres, casos_mujeres

(510658, 514356)

In [44]:
#hide
df_casos = pd.DataFrame()
df_casos["Sexo"] = ["Hombres", "Mujeres"]
df_casos["Casos confirmados"] = [casos_hombres, casos_mujeres]

In [45]:
#hide
df_casos

,Sexo,Casos confirmados
0,Hombres,510658
1,Mujeres,514356


In [46]:
#hide
def my_plot(df, last_date=last_date, precision=0):
  field = df.drop(columns='Sexo').columns[0]
  bars = alt.Chart(df).mark_bar(opacity=0.9).encode(
    x = alt.X(f"{field}"),
    y = alt.Y("Sexo"),
    color = alt.Color('Sexo'),
    tooltip = alt.Tooltip(['Sexo:N', f'{field}'])
  )
  text = alt.Chart(df).mark_text(dx=-22, dy=0, color="white").encode(
     x=alt.X(f"{field}", stack='zero'),
     y=alt.Y('Sexo'),
     text=alt.Text(f"{field}", format=f',.{precision}f')
  )
  return (bars+text).properties(
    title = f"{field} por sexo al {pd.to_datetime(last_date).strftime('%d/%m')}",
    width = 600,
    height = alt.Step(30)
  )

In [47]:
#hide
my_plot(df_casos)

alt.LayerChart(...)

In [48]:
#hide
ti_hombres = 100000*casos_hombres/pop_hombres
ti_mujeres = 100000*casos_mujeres/pop_mujeres

In [49]:
#hide
df_ti = pd.DataFrame()
df_ti["Sexo"] = ["Hombres", "Mujeres"]
df_ti["Tasa de incidencia"] = [ti_hombres, ti_mujeres]

In [50]:
#hide
my_plot(df_ti, last_date)

alt.LayerChart(...)

In [51]:
#hide
covid19_deaths_raw = pd.read_csv(
    "/Users/alonsosilva/COVID-19/DEFUNCIONES_FUENTE_DEIS_2016_2020_03072020.csv", sep=";")

In [52]:
#hide
#covid19_deaths_raw = pd.read_csv(
#    "https://raw.githubusercontent.com/alonsosilvaallende/COVID-19/master/data/DEIS_causa_COVID.csv")

In [53]:
#hide
covid19_deaths_raw = covid19_deaths_raw.T.reset_index().T

In [54]:
#hide
covid19_deaths_raw = covid19_deaths_raw.rename(
    columns={0:"año",
             1:"fecha",
             2:"género", 
             3:"edad", 
             4:"comuna_código", 
             5:"comuna", 
             6:"región", 
             7:"código_detalle",
             8:"causa_detalle",
             9:"código",
            10:"causa"})

In [55]:
#hide
deaths_last_date = (pd.to_datetime(covid19_deaths_raw["fecha"]).sort_values().iloc[-1]+pd.offsets.Day(1)).strftime("%Y-%m-%d")
deaths_last_date

'2020-07-03'

In [56]:
#hide
confirmado = covid19_deaths_raw.query("código_detalle == 'U07.1'")

In [57]:
#hide
muertes_hombres = len(confirmado.query("género == 'Hombre'"))
muertes_mujeres = len(confirmado.query("género == 'Mujer'"))
muertes_hombres, muertes_mujeres

(4090, 2967)

In [58]:
#hide
df_muertes = pd.DataFrame()
df_muertes["Sexo"] = ["Hombres", "Mujeres"]
df_muertes["Fallecimientos confirmados"] = [muertes_hombres, muertes_mujeres]

In [59]:
#hide
df_muertes

,Sexo,Fallecimientos confirmados
0,Hombres,4090
1,Mujeres,2967


In [60]:
#hide
my_plot(df_muertes, deaths_last_date)

alt.LayerChart(...)

In [61]:
#hide
tm_hombres = 100000*muertes_hombres/pop_hombres
tm_mujeres = 100000*muertes_mujeres/pop_mujeres

In [62]:
#hide
df_tm = pd.DataFrame()
df_tm["Sexo"] = ["Hombres", "Mujeres"]
df_tm["Tasa de mortalidad"] = [tm_hombres, tm_mujeres]

In [63]:
#hide
df_tm

,Sexo,Tasa de mortalidad
0,Hombres,42.608157
1,Mujeres,30.093692


In [64]:
#hide
my_plot(df_tm, deaths_last_date, 1)

alt.LayerChart(...)

In [65]:
#hide
df_cfr = pd.DataFrame()
df_cfr["Sexo"] = ["Hombres", "Mujeres"]
casos_hombres = cases_sex.loc["M","2020-07-01"]
casos_mujeres = cases_sex.loc["F","2020-07-01"]
casos_hombres, casos_mujeres

(151336, 140852)

In [66]:
#hide
cfr_hombres = 100*muertes_hombres/casos_hombres
cfr_mujeres = 100*muertes_mujeres/casos_mujeres
cfr_hombres, cfr_mujeres

(2.7025955489771105, 2.106466361854997)

In [67]:
#hide
df_cfr["Tasa de letalidad (CFR)"] = [cfr_hombres, cfr_mujeres]

In [68]:
#hide
df_cfr

,Sexo,Tasa de letalidad (CFR)
0,Hombres,2.702596
1,Mujeres,2.106466


In [69]:
#hide
my_plot(df_cfr, deaths_last_date, 2)

alt.LayerChart(...)

In [70]:
#hide_input
alt.vconcat(my_plot(df_casos, last_date), my_plot(df_muertes, deaths_last_date), my_plot(df_ti, last_date), my_plot(df_tm, deaths_last_date, 1), my_plot(df_cfr, deaths_last_date, 2))#.save(f'CC.png', scale_factor=8.0)

alt.VConcatChart(...)

In [71]:
#hide_input
display(Markdown(f" + Los hombres tienen un {100*(casos_hombres/casos_mujeres-1):.1f}% más casos confirmados y una tasa de incidencia {100*(ti_hombres/ti_mujeres-1):.1f}% mayor que las mujeres al {pd.to_datetime(last_date).strftime('%d/%m')}."))

 + Los hombres tienen un 7.4% más casos confirmados y una tasa de incidencia 2.0% mayor que las mujeres al 15/03.

In [72]:
#hide_input
display(
    Markdown(
        f" + Los hombres tienen un {100*(muertes_hombres/muertes_mujeres-1):.1f}% más fallecimientos confirmados que las mujeres y una tasa de mortalidad {100*(tm_hombres/tm_mujeres-1):.1f}% mayor que las mujeres al {pd.to_datetime('2020-06-28').strftime('%d/%m')}."))

 + Los hombres tienen un 37.8% más fallecimientos confirmados que las mujeres y una tasa de mortalidad 41.6% mayor que las mujeres al 28/06.

In [73]:
#hide_input
display(
    Markdown(
        f" + Los hombres tienen una tasa de letalidad {100*(cfr_hombres/cfr_mujeres-1):.1f}% mayor que las mujeres al {pd.to_datetime('2020-06-28').strftime('%d/%m')}.")) 

 + Los hombres tienen una tasa de letalidad 28.3% mayor que las mujeres al 28/06.

# Casos confirmados por grupo de edad

In [74]:
#hide
cases_raw = cases_raw.groupby('Grupo de edad').sum()

In [75]:
#hide
cases = pd.DataFrame()
cases['Grupo de edad'] = ['<=39', '40-49', '50-59', '60-69', '70-79', '>=80']

In [76]:
#hide
cases["Casos confirmados"] = [
    cases_raw.iloc[:8,-1].sum(),
    cases_raw.iloc[8:10,-1].sum(),
    cases_raw.iloc[10:12,-1].sum(),
    cases_raw.iloc[12:14,-1].sum(),
    cases_raw.iloc[14:16,-1].sum(),
    cases_raw.iloc[16:,-1].sum()
]

In [77]:
#hide
total_cases = cases["Casos confirmados"].sum()
total_cases

1025014

In [78]:
#hide
cases["Porcentaje"] = np.round(100*cases["Casos confirmados"]/total_cases, decimals=1)

In [79]:
#hide
cases["Texto"] = \
[f"{cases['Casos confirmados'].iloc[i]} ({cases['Porcentaje'].iloc[i]}%)" for i, val in cases.iterrows()]

In [80]:
#hide_input
bars = alt.Chart(cases.reset_index()).mark_bar(opacity=0.8, size=30).encode(
    x = alt.X('Casos confirmados'),
    y = alt.Y('Grupo de edad', sort=['<=39', '40-49', '50-59', '60-69', '70-79', '>=80']),
    tooltip = ['Grupo de edad', 'Casos confirmados', 'Porcentaje'],
    color = alt.Color('Grupo de edad', scale=alt.Scale(scheme='category10'), legend=None),
    order = alt.Order('index:O')
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Texto')
)

bars.properties(
    title = f"Covid-19 en Chile: Casos confirmados por grupo de edad al {pd.to_datetime(last_date).strftime('%d/%m/%Y')}",
    width = 600,
    height = alt.Step(40)
) + text

alt.LayerChart(...)

# Fallecidos confirmados por grupo de edad

In [81]:
#hide
deaths_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/' + \
    'master/output/producto10/FallecidosEtario.csv'
)

In [82]:
#hide
grupo_de_edad = ['<=39', '40-49', '50-59', '60-69', '70-79', '80-89', '>=90']

In [83]:
#hide
deaths = deaths_raw[['Grupo de edad', last_date]].copy()

In [84]:
#hide
total_deaths = deaths[last_date].sum()
total_deaths

21772

In [85]:
#hide
deaths = deaths.rename(columns={last_date: "Fallecidos confirmados"})

In [86]:
#hide
deaths['Porcentaje'] = np.round(100*deaths["Fallecidos confirmados"]/total_deaths, decimals=1)

In [87]:
#hide
deaths['Texto'] = \
[f"{deaths['Fallecidos confirmados'].iloc[i]} ({deaths['Porcentaje'].iloc[i]}%)" for i, val in deaths.iterrows()]

In [88]:
#hide_input
bars = alt.Chart(deaths.reset_index()).mark_bar(opacity=0.8, size=30).encode(
    x = alt.X('Fallecidos confirmados'),
    y = alt.Y('Grupo de edad', sort=grupo_de_edad),
    tooltip = ['Grupo de edad', 'Fallecidos confirmados', 'Porcentaje'],
    color = alt.Color('Grupo de edad', scale=alt.Scale(scheme='category10'), legend=None),
    order = alt.Order('index:O')
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Texto')
)

bars.properties(
    title = f"Covid-19 en Chile: Fallecidos confirmados por grupo de edad al {pd.to_datetime(last_date).strftime('%d/%m/%Y')}",
    width = 600,
    height = alt.Step(40)
) + text

alt.LayerChart(...)

# Letalidad por grupo de edad

In [89]:
#hide
data = cases.drop(columns=["Porcentaje", "Texto"])

In [90]:
#hide
a = list(deaths["Fallecidos confirmados"][:5])
a

[490, 737, 2122, 4487, 6145]

In [91]:
#hide
a.append(deaths["Fallecidos confirmados"][5:].sum())
a

[490, 737, 2122, 4487, 6145, 7791]

In [92]:
#hide
data["Fallecidos confirmados"] = a

In [93]:
#hide
data["Letalidad"] = 100*data["Fallecidos confirmados"]/data["Casos confirmados"]

In [94]:
#hide
data['Texto'] = \
[f"{data['Letalidad'].iloc[i]:.2f}" for i, val in data.iterrows()]

In [95]:
#hide_input
bars = alt.Chart(data.reset_index()).mark_bar(opacity=0.8, size=30).encode(
    x = alt.X('Letalidad'),
    y = alt.Y('Grupo de edad', sort=grupo_de_edad),
    tooltip = ['Grupo de edad', 'Casos confirmados', 'Fallecidos confirmados', 'Letalidad'],
    color = alt.Color('Grupo de edad', scale=alt.Scale(scheme='category10'), legend=None),
    order = alt.Order('index:O')
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Texto')
)

bars.properties(
    title = f"Covid-19 en Chile: Letalidad por grupo de edad al {pd.to_datetime(last_date).strftime('%d/%m/%Y')}",
    width = 600,
    height = alt.Step(40)
) + text

alt.LayerChart(...)

In [96]:
#hide
100*deaths["Fallecidos confirmados"].sum()/cases["Casos confirmados"].sum()

2.1240685493076192

In [97]:
#hide
# Población por grupo de edad

In [98]:
#hide
# censo = pd.DataFrame()
# censo['Grupo de edad'] = censo_raw['Grupo de edad']

In [99]:
#hide
# censo['Poblacion'] = censo_raw.drop(columns='Grupo de edad').sum(axis=1)

In [100]:
#hide
# new_censo = pd.DataFrame()
# new_censo['Grupo de edad'] = grupo_de_edad

In [101]:
#hide
# new_censo["Poblacion"] = \
# [censo.iloc[:8]["Poblacion"].sum(), \
# censo.iloc[8:10]["Poblacion"].sum(), \
# censo.iloc[10:12]["Poblacion"].sum(), \
# censo.iloc[12:14]["Poblacion"].sum(), \
# censo.iloc[14:16]["Poblacion"].sum(), \
# censo.iloc[16:18]["Poblacion"].sum(), \
# censo.iloc[18:]["Poblacion"].sum()]

In [102]:
#hide
# total_poblacion = new_censo["Poblacion"].sum()
# total_poblacion

In [103]:
#hide
# new_censo["Porcentaje"] = np.round(100*new_censo["Poblacion"]/total_poblacion, decimals=1)

In [104]:
#hide
# new_censo['Texto'] = \
# [f"{format(new_censo['Poblacion'].iloc[i],',').replace(',', '.')} ({new_censo['Porcentaje'].iloc[i]}%)" for i, val in new_censo.iterrows()]

In [105]:
#hide
# bars = alt.Chart(new_censo.reset_index()).mark_bar(opacity=0.8, size=30).encode(
#     x = alt.X('Poblacion', axis=alt.Axis(title="Población")),
#     y = alt.Y('Grupo de edad', sort=grupo_de_edad),
#     tooltip = ['Grupo de edad', 'Poblacion', 'Porcentaje'],
#     color = alt.Color('Grupo de edad', scale=alt.Scale(scheme='category10'), legend=None),
#     order = alt.Order('index:O')
# )

# text = bars.mark_text(
#     align='left',
#     baseline='middle',
#     dx=3  # Nudges text to right so it doesn't appear on top of the bar
# ).encode(
#     text=alt.Text('Texto')
# )

# bars.properties(
#     title = f"Población Censo 2017",
#     width = 600,
#     height = alt.Step(40)
# ) + text

# Pacientes UCI por rango de edad

In [106]:
#hide
pacientes_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/'
    + 'master/output/producto9/HospitalizadosUCIEtario.csv'
)

In [107]:
#hide
pacientes = pacientes_raw[['Grupo de edad', last_date]]

In [108]:
#hide
total_pacientes = pacientes[last_date].sum()
total_pacientes

1981

In [109]:
#hide
pacientes = pacientes.rename(columns={last_date: "Pacientes UCI"})

In [110]:
#hide
pacientes['Porcentaje'] = np.round(100*pacientes["Pacientes UCI"]/total_pacientes, decimals=1)

In [111]:
#hide
pacientes['Texto'] = \
[f"{pacientes['Pacientes UCI'].iloc[i]} ({pacientes['Porcentaje'].iloc[i]}%)" for i, val in pacientes.iterrows()]

In [112]:
#hide_input
bars = alt.Chart(pacientes.reset_index()).mark_bar(opacity=0.8, size=30).encode(
    x = alt.X('Pacientes UCI'),
    y = alt.Y('Grupo de edad', sort=grupo_de_edad),
    order = alt.Order('index:O'),
    tooltip = ['Grupo de edad', 'Pacientes UCI', 'Porcentaje'],
    color = alt.Color('Grupo de edad', scale=alt.Scale(scheme='category10'), legend=None)
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Texto')
)

bars.properties(
    title = f"Pacientes UCI presentes el {pd.to_datetime(last_date).strftime('%d/%m/%Y')}",
    width = 600,
    height = alt.Step(40)
) + text

alt.LayerChart(...)

# Evolución de casos confirmados por rango de edad

In [113]:
#hide
cases_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/' \
    + 'master/output/producto16/CasosGeneroEtario.csv'
)

In [114]:
#hide
cases = pd.DataFrame()
cases['<=39'] = cases_raw.groupby('Grupo de edad').sum().iloc[:8].sum()
cases['40-49'] = cases_raw.groupby('Grupo de edad').sum().iloc[8:10].sum()
cases['50-59'] = cases_raw.groupby('Grupo de edad').sum().iloc[10:12].sum()
cases['60-69'] = cases_raw.groupby('Grupo de edad').sum().iloc[12:14].sum()
cases['70-79'] = cases_raw.groupby('Grupo de edad').sum().iloc[14:16].sum()
cases['>=80'] = cases_raw.groupby('Grupo de edad').sum().iloc[16:].sum()

In [115]:
#hide
cases = cases.T.reset_index()
cases = cases.rename(columns={'index': 'Grupo de edad'})

In [116]:
#hide
cases = cases.reset_index().melt(['index', 'Grupo de edad'], var_name="fecha (año-mes-día)", value_name="casos confirmados")

In [117]:
#hide_input
input_dropdown = alt.binding_select(options=data['Grupo de edad'].unique())
selection1 = alt.selection_single(fields=['Grupo de edad'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['Grupo de edad'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('Grupo de edad:N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

bars = alt.Chart(cases).mark_bar().encode(
    x = 'fecha (año-mes-día):N',
    y = 'casos confirmados:Q',
    color = color,
    tooltip = ['fecha (año-mes-día)', 'Grupo de edad', 'casos confirmados'],
    order=alt.Order(
    # Sort the segments of the bars by this field
    'index',
    sort='descending'
    )
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

legend = alt.Chart(cases).mark_point().encode(
    y=alt.Y('Grupo de edad', axis=alt.Axis(orient='right'), sort=grupo_de_edad),
    color=color
).add_selection(
    selection1, selection2
)

bars.properties(
    title = 'COVID-19 en Chile: Evolución de casos confirmados por rango de edad',
    width = 600
) | legend

alt.HConcatChart(...)

In [118]:
#hide_input
selection2 = alt.selection_multi(fields=['Grupo de edad'], on='mouseover')

color = alt.condition(selection2,
                    alt.Color('Grupo de edad:N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

bars = alt.Chart(cases).mark_area().encode(
    x = 'fecha (año-mes-día):N',
    y = alt.Y('casos confirmados:Q', stack="normalize"),
    color = color,
    tooltip = ['fecha (año-mes-día)', 'Grupo de edad', 'casos confirmados'],
    order=alt.Order(
    # Sort the segments of the bars by this field
    'index',
    sort='descending'
    )
).add_selection(
    selection2
)

legend = alt.Chart(cases).mark_point().encode(
    y=alt.Y('Grupo de edad', axis=alt.Axis(orient='right'), sort=grupo_de_edad),
    color=color
).add_selection(
    selection2
)

bars.properties(
    title = 'COVID-19 en Chile: Evolución de casos confirmados por rango de edad',
    width = 600
) | legend

alt.HConcatChart(...)

# Evolución de fallecidos confirmados por rango de edad

In [21]:
#hide
data = deaths_raw.reset_index().melt(['index', 'Grupo de edad'])

NameError: name 'deaths_raw' is not defined

In [ ]:
#hide
data = data.rename(columns={'variable': "fecha (año-mes-día)", 'value': "fallecidos confirmados"})

In [ ]:
#hide_input
input_dropdown = alt.binding_select(options=data['Grupo de edad'].unique())
selection1 = alt.selection_single(fields=['Grupo de edad'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['Grupo de edad'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('Grupo de edad:N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

bars = alt.Chart(data).mark_bar().encode(
    x = 'fecha (año-mes-día):N',
    y = 'fallecidos confirmados:Q',
    color = color,
    tooltip = ['fecha (año-mes-día)', 'Grupo de edad', 'fallecidos confirmados'],
    order=alt.Order(
    # Sort the segments of the bars by this field
    'index',
    sort='descending'
    )
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

legend = alt.Chart(data).mark_point().encode(
    y=alt.Y('Grupo de edad', axis=alt.Axis(orient='right'), sort=grupo_de_edad),
    color=color
).add_selection(
    selection1, selection2
)

bars.properties(
    title = 'COVID-19 en Chile: Evolución de fallecidos confirmados por rango de edad',
    width = 600
) | legend

In [ ]:
#hide_input
selection2 = alt.selection_multi(fields=['Grupo de edad'], on='mouseover')

color = alt.condition(selection2,
                    alt.Color('Grupo de edad:N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

bars = alt.Chart(data).mark_area().encode(
    x = 'fecha (año-mes-día):N',
    y = alt.Y('fallecidos confirmados:Q', stack="normalize", sort=grupo_de_edad),
    color = color,
    tooltip = ['fecha (año-mes-día)', 'Grupo de edad', 'fallecidos confirmados'],
    order=alt.Order(
    # Sort the segments of the bars by this field
    'index',
    sort='descending'
    )
).add_selection(
    selection2
)

legend = alt.Chart(data).mark_point().encode(
    y=alt.Y('Grupo de edad',  axis=alt.Axis(orient='right'), sort=grupo_de_edad),
    color=color
).add_selection(
    selection2
)

bars.properties(
    title = 'COVID-19 en Chile: Evolución de fallecidos confirmados normalizados por rango de edad',
    width = 600
) | legend

# Evolución de la letalidad por rango de edad

In [ ]:
#hide
deaths = deaths_raw.iloc[:5]
deaths = deaths.append(deaths_raw.iloc[5:].sum(), ignore_index=True)
deaths.loc[5, "Grupo de edad"] = ">=80"

In [ ]:
#hide
cases = pd.DataFrame()
cases['<=39'] = cases_raw.groupby('Grupo de edad').sum().iloc[:8].sum()
cases['40-49'] = cases_raw.groupby('Grupo de edad').sum().iloc[8:10].sum()
cases['50-59'] = cases_raw.groupby('Grupo de edad').sum().iloc[10:12].sum()
cases['60-69'] = cases_raw.groupby('Grupo de edad').sum().iloc[12:14].sum()
cases['70-79'] = cases_raw.groupby('Grupo de edad').sum().iloc[14:16].sum()
cases['>=80'] = cases_raw.groupby('Grupo de edad').sum().iloc[16:].sum()

In [ ]:
#hide
cases = cases.T.reset_index()
cases = cases.rename(columns={'index': 'Grupo de edad'})

In [ ]:
#hide
deaths = deaths.set_index("Grupo de edad").T
cases = cases.set_index("Grupo de edad").T
letalidad = deaths.divide(cases['2020-04-09':])

In [ ]:
#hide
cases.tail(3)

In [ ]:
#hide
deaths.tail(3)

In [ ]:
#hide
letalidad = letalidad.reset_index().melt("index")
letalidad = letalidad.rename(columns={"index": "fecha (año-mes-día)", "value": "letalidad"})
letalidad = letalidad.dropna()

In [ ]:
#hide
label = alt.selection_single(
    encodings=['x'], # limit selection to x-axis value
    on='mouseover',  # select on mouseover events
    nearest=True,    # select data point nearest the cursor
    empty='none'     # empty selection includes no data points
)

input_dropdown = alt.binding_select(options=letalidad['Grupo de edad'].unique())
selection1 = alt.selection_single(fields=['Grupo de edad'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['Grupo de edad'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('Grupo de edad:N', scale=alt.Scale(scheme='tableau10'), legend=None),
                    alt.value('lightgray'))


base = alt.Chart(letalidad).mark_line().encode(
    x = 'fecha (año-mes-día):N',
    y = alt.Y('letalidad:Q', sort=['<=39', '40-49', '50-59', '60-69', '70-79', '>=80'], axis=alt.Axis(format='%')),
    color = color,
    tooltip = ['fecha (año-mes-día)', 'Grupo de edad', 'letalidad'],
).add_selection(
    label, selection1, selection2
).transform_filter(
    selection1
)

chart = alt.layer(
    base, # base line chart
    
    # add a rule mark to serve as a guide line
    alt.Chart().mark_rule(color='#aaa').encode(
        x = alt.X('fecha (año-mes-día):N', axis=alt.Axis(title='fecha (año-mes-día):N'), sort=None)
    ).transform_filter(label),
    
    # add circle marks for selected time points, hide unselected points
    base.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
    ).add_selection(label),
    
    # add white stroked text to provide a legible background for labels
    base.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
        text='letalidad:Q'
    ).transform_filter(label),
    
    # add text labels for stock prices
    base.mark_text(align='left', dx=5, dy=-5).encode(
        text='letalidad:Q'
    ).transform_filter(label),
    
    data=letalidad
)

legend = alt.Chart(letalidad).mark_point().encode(
    y=alt.Y('Grupo de edad', axis=alt.Axis(orient='right'), sort=['>=80','70-79','60-69','50-59','40-49','<=39']),
    color=color
).add_selection(
    selection1, selection2
)

chart.properties(
    title = 'COVID-19 en Chile: Evolución de la letalidad por rango de edad',
    width = 600
) | legend

In [ ]:
#hide_input
label = alt.selection_single(
    encodings=['x'], # limit selection to x-axis value
    on='mouseover',  # select on mouseover events
    nearest=True,    # select data point nearest the cursor
    empty='none'     # empty selection includes no data points
)

base = alt.Chart(letalidad).mark_line(point=True).encode(
    x = 'fecha (año-mes-día):N',
    y = alt.Y('letalidad:Q', sort=['<=39', '40-49', '50-59', '60-69', '70-79', '>=80'], axis=alt.Axis(format='%')),
    color = alt.Color('Grupo de edad:N', scale=alt.Scale(scheme='tableau10'), legend=None),
    #tooltip = ['fecha (año-mes-día)', 'Grupo de edad', 'letalidad'],
)

legend = alt.Chart(letalidad).mark_point().encode(
    y=alt.Y('Grupo de edad', axis=alt.Axis(orient='right'), sort=['>=80','70-79','60-69','50-59','40-49','<=39']),
    color=alt.Color('Grupo de edad:N', scale=alt.Scale(scheme='tableau10'), legend=None)
)

alt.layer(
    base, # base line chart
    
    # add a rule mark to serve as a guide line
    alt.Chart().mark_rule(color='#aaa').encode(
        x = alt.X('fecha (año-mes-día):N', axis=alt.Axis(title='fecha (año-mes-día)'), sort=None)
    ).transform_filter(label),
    
    # add circle marks for selected time points, hide unselected points
    base.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
    ).add_selection(label),
    
    # add white stroked text to provide a legible background for labels
    base.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
        text='letalidad:Q'
    ).transform_filter(label),
    
    # add text labels for stock prices
    base.mark_text(align='left', dx=5, dy=-5).encode(
        text='letalidad:Q'
    ).transform_filter(label),
    
    data=letalidad
).properties(
    title = 'COVID-19 en Chile: Evolución de la letalidad por rango de edad',
    width = 600
) | legend

# legend = alt.Chart(letalidad).mark_point().encode(
#     y=alt.Y('Grupo de edad', axis=alt.Axis(orient='right'), sort=['>=80','70-79','60-69','50-59','40-49','<=39']),
#     color=color
# )

# chart.properties(
#     title = 'COVID-19 en Chile: Evolución de la letalidad por rango de edad',
#     width = 600
# ) | legend